# 畳み込み層のクラスを実装する

In [1]:
import numpy as np

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/commonフォルダのutil.pyを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')

from common.util import im2col, col2im

Google Colab. 上での実行です
「ファイルを選択」から、notebook/commonフォルダのutil.pyを選択し、アップロードしてください


Saving util.py to util.py


### [演習]
* 以下のConvolutionクラスを完成させましょう

In [2]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W # フィルターの重み(配列形状:フィルターの枚数, チャンネル数, フィルターの高さ, フィルターの幅)
        self.b = b #フィルターのバイアス
        self.stride = stride # ストライド数
        self.pad = pad # パディング数
        
        # インスタンス変数の宣言
        self.x = None   
        self.col = None
        self.col_W = None
        self.dcol = None
        self.dW = None
        self.db = None

    def forward(self, x):
        """
        順伝播計算
        x : 入力(配列形状=(データ数, チャンネル数, 高さ, 幅))
        """
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = (H + 2*self.pad - FH) // self.stride + 1 # 出力の高さ(端数は切り捨てる)
        out_w =(W + 2*self.pad - FW) // self.stride + 1# 出力の幅(端数は切り捨てる)

        # 畳み込み演算を効率的に行えるようにするため、入力xを行列colに変形する
        col = im2col(x, FH, FW, self.stride, self.pad)
        
        # 重みフィルターを2次元配列に変形する
        # col_Wの配列形状は、(C*FH*FW, フィルター枚数)
        col_W = self.W.reshape(FN, -1).T

        # 行列の積を計算し、バイアスを足す
        out = np.dot(col, col_W) + self.b
        
        # 画像形式に戻して、チャンネルの軸を2番目に移動する
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        self.x = x
        self.col = col
        self.col_W = col_W

        return out

    def backward(self, dout):
        """
        逆伝播計算
        Affineレイヤと同様の考え方で、逆伝播させる
        dout : 出力層側から伝わってきた勾配(配列形状=(データ数, チャンネル数, 高さ, 幅))
        return : 入力層側へ伝える勾配
        """
        FN, C, FH, FW = self.W.shape
        
        # doutのチャンネル数軸を4番目に移動させ、2次元配列に変形する
        # doutの列数は、チャンネル数(=フィルター数)になる
        # doutの行数は、データ数*doutの高さ*doutの幅になる        
        dout = dout.transpose(0,2,3,1).reshape(-1, FN)

        # バイアスbは、doutのチャンネル毎に、(データ数*doutの高さ*doutの幅)個の要素を足し合わせる
        self.db = np.sum(dout, axis=0)
        
        # dWは、入力行列colと行列doutの積になる
        self.dW = np.dot(self.col.T, dout)
        
        # dWを(フィルター数, チャンネル数, フィルター高さ、フィルター幅)の配列形状に変形する
        self.dW = self.dW.transpose(1, 0).reshape(FN, C, FH, FW)

        # 入力側の勾配は、doutにフィルターの重みを掛けて求める
        dcol = np.dot(dout, self.col_W.T)
        
        # 勾配を4次元配列(データ数, チャンネル数, 高さ, 幅)に変形する
        dx = col2im(dcol, self.x.shape, FH, FW, self.stride, self.pad, is_backward=True)

        self.dcol = dcol # 結果を確認するために保持しておく
            
        return dx

In [3]:
# 動作確認 条件1
channel = 1
filter_num = 2
filter_size = 2
N = 2
stride = 2
pad = 0
input_size = 4

# # 動作確認 条件2
# channel = 1
# filter_num = 2
# filter_size = 2
# N = 1
# stride = 1
# pad = 1
# input_size = 2


x =  np.random.randn(N, channel, input_size, input_size)
W =  np.random.randn(filter_num, channel, filter_size, filter_size)
b =  np.random.randn(filter_num)

print("x=")
print(x.round(2))
print("W=")
print(W.round(2))
print("b=")
print(b.round(2))
print()

# Convolutionクラスのインスタンスを生成
cv = Convolution(W, b, stride=stride, pad=pad)

print("順伝播計算")
out = cv.forward(x)
print("col=")
print(cv.col.round(2))
print("col_W=")
print(cv.col_W.round(2))
print("out=")
print(out.round(2))
print()

print("逆伝播計算")
out_h = (input_size + 2*pad - filter_size) // stride + 1 # 出力の高さ
out_w =(input_size + 2*pad - filter_size) // stride + 1# 出力の幅
dout =  np.random.randn(N, filter_num, out_h, out_w)
print("dout=")
print(dout.round(2))
dx = cv.backward(dout)
print("dcol=")
print(cv.dcol.round(2))
print("dx=")
print(dx.round(2))
print()


x=
[[[[ 0.14 -0.41 -0.97 -0.38]
   [ 1.55 -1.08 -0.79 -0.02]
   [-0.3  -0.03  0.1  -0.17]
   [ 0.24  0.86  0.57  0.72]]]


 [[[-0.71 -0.76 -0.72 -0.86]
   [ 1.55 -0.22  1.64 -0.82]
   [ 1.07 -0.52  0.21 -0.02]
   [ 0.05 -0.05  0.45  1.59]]]]
W=
[[[[-0.56 -2.09]
   [ 1.21  0.84]]]


 [[[ 0.75 -1.61]
   [-0.22  1.91]]]]
b=
[-0.48 -0.53]

順伝播計算
col=
[[ 0.14 -0.41  1.55 -1.08]
 [-0.97 -0.38 -0.79 -0.02]
 [-0.3  -0.03  0.24  0.86]
 [ 0.1  -0.17  0.57  0.72]
 [-0.71 -0.76  1.55 -0.22]
 [-0.72 -0.86  1.64 -0.82]
 [ 1.07 -0.52  0.05 -0.05]
 [ 0.21 -0.02  0.45  1.59]]
col_W=
[[-0.56  0.75]
 [-2.09 -1.61]
 [ 1.21 -0.22]
 [ 0.84  1.91]]
out=
[[[[ 1.26 -0.11]
   [ 0.75  1.11]]

  [[-2.19 -0.52]
   [ 0.88  1.07]]]


 [[[ 3.2   3.03]
   [ 0.04  1.32]]

  [[-0.6  -1.6 ]
   [ 1.01  2.6 ]]]]

逆伝播計算
dout=
[[[[ 0.03 -0.3 ]
   [-0.8   0.75]]

  [[ 0.14  0.3 ]
   [ 0.33 -0.07]]]


 [[[ 0.07 -0.75]
   [ 0.76  1.74]]

  [[-0.59 -0.18]
   [-0.45 -1.2 ]]]]
dcol=
[[ 0.09 -0.29 -0.    0.3 ]
 [ 0.4   0.14 -0.43  